Validate modle code

In [2]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import numpy as np


def division_function(n, d):
    if d:
        return n / d
    elif n == 0 and d == 0:
        return 0
    else:
        return None


def validate_model(model, X, Y, fold):
    """
    validates the model with a k-fold validation which is iterated
    returns the mean accuracy, specificiy, recall, precision, f1 score and auc score
    """

    splits = 5
    iteration = 10

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = []
    f1_list = []

    if fold == "Strat":
        folds = StratifiedKFold(n_splits=splits)
    elif fold == "K":
        folds = KFold(splits, shuffle=True)

    # Iterate "interation" times of k-fold
    for i in range(1, iteration):
        # print(f'Iteration {i}/{iteration}')

        acc_total = 0
        specificity_total = 0
        recall_total = 0
        precision_total = 0
        f1_total = 0

        for train_index, test_index in folds.split(X, Y):
            x_train = X.iloc[train_index, :]
            x_test = X.iloc[test_index, :]
            y_train = Y.iloc[train_index, :]
            y_test = Y.iloc[test_index, :]

            # scale
            sc = MinMaxScaler()
            x_train = sc.fit_transform(x_train)
            x_test = sc.transform(x_test)

            # fit model and predict
            model.fit(x_train, np.ravel(y_train))
            y_pred = model.predict(x_test)

            conf_matrix = confusion_matrix(y_test, y_pred)
            TN = conf_matrix[0][0]
            FP = conf_matrix[0][1]
            FN = conf_matrix[1][0]
            TP = conf_matrix[1][1]

            accuracy = (division_function((TP + TN), (TP + TN + FP + FN))) * 100
            specificity = division_function(TN, (TN + FP)) * 100
            recall = division_function(TP, (TP + FN)) * 100  # recall
            precision = division_function(TP, (TP + FP)) * 100
            f1_score = division_function(2 * (recall * precision), (recall + precision))

            # sum it up
            acc_total += accuracy
            specificity_total += specificity
            recall_total += recall
            precision_total += precision
            f1_total += f1_score

        # avg
        accuracy_mean = acc_total / splits
        recall_mean = recall_total / splits
        specificity_mean = specificity_total / splits
        precision_mean = precision_total / splits
        f1_mean = f1_total / splits

        acc_list.append(accuracy_mean)
        recall_list.append(recall_mean)
        specificity_list.append(specificity_mean)
        precision_list.append(precision_mean)
        f1_list.append(f1_mean)

    return (
        np.mean(acc_list),
        np.mean(specificity_list),
        np.mean(recall_list),
        np.mean(precision_list),
        np.mean(f1_list)
    )


Evaluate modle code

In [14]:
def evaluate_model(model, x_train, x_test, y_train, y_test):
    sc = MinMaxScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    accuracy = (division_function((TP + TN), (TP + TN + FP + FN))) * 100
    specificity = division_function(TN, (TN + FP)) * 100
    precision = division_function(TP, (TP + FP)) * 100
    recall = division_function(TP, (TP + FN)) * 100  # recall
    f1_score = division_function(2 * (recall * precision), (recall + precision))

    return accuracy, specificity, recall, precision, f1_score

Model

In [23]:
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  
from sklearn.model_selection import train_test_split
from feature_reduction import feature_reduction_lda, feature_reduction_mrmr
from imblearn.over_sampling import SMOTE
path = '/Users/athena.kam/Documents/Thesis/codebase/thesis-2023-athena'
os.chdir(path)

CV_SPLIT = 5

def get_best_param_KNN(x_train, y_train):
    param_grid = {
        "n_neighbors": range(1, 4, 2),
        "weights": ['uniform', 'distance'],
        "metric": ['euclidean', 'manhattan', 'minkowski'],
    }
    grid = GridSearchCV(
        KNeighborsClassifier(),
        param_grid,
        refit=True,
        verbose=0,
        return_train_score=True,
        cv=CV_SPLIT,
    )
    grid.fit(x_train, y_train)
    print(grid.best_estimator_.get_params())
    return grid


def train_test_KNN(filename:str,hold_out:bool = True,include_personal_q:bool = False , grid_search:bool = True,reduce:bool = True,over_sample:bool = False,model_weights:dict = {},random_state:int = 0):

    df = pd.read_csv(filename)

    if include_personal_q:
        df = df[df['noPersonalQ']!=1].reset_index(drop=True)
    else:
        df = df[df['personalQ']!=1].reset_index(drop=True)
    
    headers = df.columns
    non_embeddings_headers = []
    
    for header in headers:
        if header.find('embbedings')<0:
            non_embeddings_headers.append(header)

    X = df.drop(columns=non_embeddings_headers)
    Y = df['classification']

    # Feature Reduction 
    if reduce:
        #x_train = feature_reduction_pca(x_train,0.9).values
        #X = feature_reduction_lda(X,Y)
        X = feature_reduction_mrmr(X,Y,20)
        
    else:
        x_val = X.values
        X = StandardScaler().fit_transform(x_val)
    
    # Test Train split
    if hold_out:
        x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.20,random_state=random_state)

        # Oversample on training set
        if over_sample:
            sm = SMOTE(random_state=12)
            x_train,y_train = sm.fit_resample(x_train, y_train) 
    else:
        x_train = X
        y_train = Y


    if grid_search: 
        grid = get_best_param_KNN(x_train=x_train,y_train=y_train) 
        model_svc = grid.best_estimator_
    else:
        model_svc = KNeighborsClassifier(C = model_weights['C'],gamma=model_weights['gamma'],kernel=model_weights['kernel'])
        
    accuracy, specificiy, recall, precision, f1_score =validate_model(model_svc,pd.DataFrame(x_train),pd.DataFrame(y_train),"Strat")
    print(f'\tAverage Accuracy: {accuracy} \n\
      Average Specificity: {specificiy} \n\
      Average Recall: {recall}\n\
      Average Precision:{precision}\n\
      Average F1 score {f1_score}\n\
      ')
    
    if hold_out:
        accuracy, specificiy, recall, precision, f1_score =evaluate_model(model_svc,x_train,x_test,y_train,y_test)
        print('____________________________________________')
        print('Evaluate model')
        print(f'\tAccuracy: {accuracy} \n\
        Specificity: {specificiy} \n\
        Recall: {recall}\n\
        Precision:{precision}\n\
        F1 score {f1_score}\n\
        ')
        return accuracy, specificiy, recall, precision, f1_score
    else: 
        return accuracy, specificiy, recall, precision, f1_score


Testing different seeds (Hold out only)

In [4]:
def train_test_TEST_avg_seeds(
    filename: str,
    hold_out: bool = True,
    include_personal_q: bool = False,
    grid_search: bool = True,
    reduce: bool = True,
    model_weights: dict = {},
    over_sample: bool = False,
):
    random_states = [0, 5, 13, 27, 36, 42]
    n_states = len(random_states)

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = []
    f1_list = []

    for random_state in random_states:
        print(f"Random State: {random_state}")
        accuracy, specificity, recall, precision, f1_score = train_test_KNN(
            filename=filename,
            hold_out=hold_out,
            include_personal_q=include_personal_q,
            grid_search=grid_search,
            reduce=reduce,
            model_weights=model_weights,
            random_state=random_state,
            over_sample=over_sample,
        )
        acc_list.append(accuracy)
        specificity_list.append(specificity)
        recall_list.append(recall)
        precision_list.append(precision)
        f1_list.append(f1_score)

    accuracy_mean = np.mean(acc_list)
    specificity_mean = np.mean(specificity_list)
    recall_mean = np.mean(recall_list)
    precision_mean = np.mean(precision_list)
    f1_mean = np.mean(f1_list)

    print("Accuracy list: ", acc_list)
    print("Specificity list: ", specificity_list)
    print("Recall list: ", recall_list)
    print("Precision list: ", precision_list)
    print("F1  score list: ", f1_list)

    print(
        f"\tAverage Accuracy: {accuracy_mean} \n\
      Average Specificity: {specificity_mean} \n\
      Average Recall: {recall_mean}\n\
      Average Precision:{precision_mean}\n\
      Average F1 score {f1_mean}\n\
      "
    )

In [24]:
train_test_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=True)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  8.77it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 95.0 
      Average Recall: 100.0
      Average Precision:95.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 62.5 
        Specificity: 40.0 
        Recall: 100.0
        Precision:50.0
        F1 score 66.66666666666667
        
Random State: 5


100%|██████████| 20/20 [00:02<00:00,  8.73it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
	Average Accuracy: 90.95238095238096 
      Average Specificity: 81.66666666666666 
      Average Recall: 100.0
      Average Precision:86.0
      Average F1 score 92.06349206349206
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 83.33333333333334 
        Recall: 100.0
        Precision:66.66666666666666
        F1 score 80.0
        
Random State: 13


...

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 81.42857142857142 
      Average Specificity: 80.0 
      Average Recall: 81.66666666666666
      Average Precision:85.33333333333333
      Average F1 score 82.66666666666667
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 36


100%|██████████| 20/20 [00:02<00:00,  9.81it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
	Average Accuracy: 88.57142857142857 
      Average Specificity: 78.33333333333333 
      Average Recall: 100.0
      Average Precision:83.0
      Average F1 score 89.92063492063491
      
____________________________________________
Evaluate model
	Accuracy: 75.0 
        Specificity: 80.0 
        Recall: 66.66666666666666
        Precision:66.66666666666666
        F1 score 66.66666666666666
        
Random State: 42


100%|██████████| 20/20 [00:02<00:00,  8.74it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 86.66666666666667 
      Average Specificity: 79.99999999999999 
      Average Recall: 93.33333333333333
      Average Precision:87.0
      Average F1 score 88.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 85.71428571428571 
        Recall: 100.0
        Precision:50.0
        F1 score 66.66666666666667
        
Accuracy list:  [62.5, 87.5, 75.0, 100.0, 75.0, 87.5]
Specificity list:  [40.0, 83.33333333333334, 100.0, 100.0, 80.0, 85.71428571428571]
Recall list:  [100.0, 100.0, 71.42857142857143, 100.0, 66.66666666666666, 100.0]
Precision list:  [50.0, 66.66666666666666, 100.0, 100.0, 66.66666666666666, 50.0]
F1  score list:  [66.66666666666667, 80.0, 83.33333333333333, 100.0, 66.66666666666666, 66.66666666666667]
	Average Accuracy: 81.25 
      Aver

In [ ]:
train_test_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=False)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  8.45it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 88.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 90.0
      Average Precision:86.66666666666666
      Average F1 score 85.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 83.33333333333334 
        Recall: 100.0
        Precision:83.33333333333334
        F1 score 90.9090909090909
        
Random State: 5


100%|██████████| 20/20 [00:01<00:00, 10.08it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 92.0 
      Average Specificity: 100.0 
      Average Recall: 83.33333333333333
      Average Precision:100.0
      Average F1 score 89.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 76.0 
      Average Specificity: 80.0 
      Average Recall: 76.66666666666666
      Average Precision:80.0
      Average F1 score 71.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 36


100%|██████████| 20/20 [00:02<00:00,  9.22it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 96.0 
      Average Specificity: 100.0 
      Average Recall: 90.0
      Average Precision:100.0
      Average F1 score 93.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 100.0 
        Recall: 80.0
        Precision:100.0
        F1 score 88.88888888888889
        
Random State: 42


100%|██████████| 20/20 [00:02<00:00,  9.12it/s]


{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
	Average Accuracy: 84.0 
      Average Specificity: 86.66666666666666 
      Average Recall: 80.0
      Average Precision:83.33333333333333
      Average F1 score 79.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 87.5 
        Recall: 100.0
        Precision:75.0
        F1 score 85.71428571428571
        
Accuracy list:  [90.9090909090909, 100.0, 45.45454545454545, 100.0, 90.9090909090909, 90.9090909090909]
Specificity list:  [83.33333333333334, 100.0, 100.0, 100.0, 100.0, 87.5]
Recall list:  [100.0, 100.0, 25.0, 100.0, 80.0, 100.0]
Precision list:  [83.33333333333334, 100.0, 100.0, 100.0, 100.0, 75.0]
F1  score list:  [90.9090909090909, 100.0, 40.0, 100.0, 88.88888888888889, 85.71428571428571]
	Average Accuracy: 86.36363636363636 
      Average Specifici